![Scikit-Learn Image](https://s3-us-west-2.amazonaws.com/gmedasani-chicago-escape-conference/setup/images/scikit-learn-logo-small.png)
![Machine Learning Image](https://s3-us-west-2.amazonaws.com/gmedasani-chicago-escape-conference/setup/images/machine-learning.png)


# Machine Learing - Linear Regression

This section covers a commom supervised learning pipeline, using a subset of the [Million Song Dataset](http://labrosa.ee.columbia.edu/millionsong/) from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/YearPredictionMSD). Our goal is to train a linear regression model to predict the release year of a song given a set of audio features.


This exercise will cover: 

* Part 1: Explore the dataset
* Part 2: Feature Engineering
* Part 3: Create and evaluate a baseline model
* Part 4: Create a Linear Regression Model
* Part 5: Choose the best model by Hyperparamter tuning


Note that, for reference, you can look up the details of the relevant Scikit learn methods in [Scikit-Learn](http://scikit-learn.org/stable/) and the relevant NumPy methods in the [NumPy Reference](http://docs.scipy.org/doc/numpy/reference/index.html)

## Part 1: Explore the dataset

### Initial setup

* Check if spark context is available
* Import needed libraries
* Define the dataset path

In [1]:
import os.path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Set the value of the studentid to your studentid provided in the instructions'
studentid = 'student1'
filepath = '/home/'+studentid+'/2016-escape-conference-chicago/datasets/millionsong.txt'
print filepath

/home/student1/2016-escape-conference-chicago/datasets/millionsong.txt


### Load the dataset

* Create a millionsongs dataframe
* Explore the millionsongs dataframe

In [4]:
millionsongs_raw_df = pd.read_csv(filepath,header=None)

In [5]:
millionsongs_raw_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,2001.0,0.884124,0.610454,0.600498,0.474669,0.247233,0.357306,0.344136,0.339641,0.600859,0.425705,0.604915,0.419193
1,2001.0,0.854412,0.604125,0.593634,0.495885,0.266308,0.261472,0.506387,0.464454,0.665799,0.542969,0.580444,0.445219
2,2001.0,0.908983,0.632063,0.557429,0.498264,0.276396,0.312810,0.448530,0.448674,0.649791,0.489869,0.591908,0.450002
3,2001.0,0.842525,0.561827,0.508715,0.443531,0.296734,0.250214,0.488541,0.360509,0.575435,0.361006,0.678379,0.409037
4,2001.0,0.909303,0.653608,0.585581,0.473251,0.251417,0.326977,0.404323,0.371155,0.629402,0.482243,0.566901,0.463374


In [35]:
millionsongs_raw_df.count()

0     6724
1     6724
2     6724
3     6724
4     6724
5     6724
6     6724
7     6724
8     6724
9     6724
10    6724
11    6724
12    6724
dtype: int64

In [36]:
millionsongs_raw_df.columns = ['year','f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12']

In [37]:
millionsongs_raw_df.head()

,year,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12
0,2001.0,0.884124,0.610454,0.600498,0.474669,0.247233,0.357306,0.344136,0.339641,0.600859,0.425705,0.604915,0.419193
1,2001.0,0.854412,0.604125,0.593634,0.495885,0.266308,0.261472,0.506387,0.464454,0.665799,0.542969,0.580444,0.445219
2,2001.0,0.908983,0.632063,0.557429,0.498264,0.276396,0.312810,0.448530,0.448674,0.649791,0.489869,0.591908,0.450002
3,2001.0,0.842525,0.561827,0.508715,0.443531,0.296734,0.250214,0.488541,0.360509,0.575435,0.361006,0.678379,0.409037
4,2001.0,0.909303,0.653608,0.585581,0.473251,0.251417,0.326977,0.404323,0.371155,0.629402,0.482243,0.566901,0.463374


In [38]:
millionsongs_raw_df.dtypes

year    float64
f1      float64
f2      float64
f3      float64
f4      float64
f5      float64
f6      float64
f7      float64
f8      float64
f9      float64
f10     float64
f11     float64
f12     float64
dtype: object

In [39]:
millionsongs_raw_df.describe()

,year,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12
count,6724.000000,6724.000000,6724.000000,6724.000000,6724.000000,6724.000000,6724.000000,6724.000000,6724.000000,6724.000000,6724.000000,6724.000000,6724.000000
mean,1975.785098,0.661996,0.533941,0.469804,0.437685,0.275382,0.438947,0.432559,0.469407,0.579688,0.500435,0.514644,0.483023
std,21.419860,0.156361,0.126834,0.093941,0.080921,0.065018,0.121503,0.080623,0.090353,0.076709,0.105894,0.100386,0.091144
min,1922.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1960.000000,0.564927,0.462435,0.414965,0.392360,0.234428,0.354939,0.388095,0.414316,0.538227,0.431807,0.455809,0.425435
50%,1977.000000,0.678626,0.554613,0.471210,0.435271,0.277502,0.421595,0.441366,0.467141,0.581490,0.496960,0.518566,0.478539
75%,1994.000000,0.779215,0.623506,0.525948,0.480047,0.316274,0.508058,0.485982,0.523495,0.624050,0.565828,0.577818,0.537053
max,2011.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Part 2: Feature Engineering

As part of the feature enginering, we will perform following two steps

* Shift the labels(year)
* Create Interaction features and polynomial features
* Create Train, Test and Validation datasets

#### Shift Lables

As we just saw, the lables are years in the 1900s and 2000s. In learning problems, it is often natural to shift lables such that they start from zero. 

In [40]:
minYear = millionsongs_raw_df.year.min()
print 'Minimum year is : '+str(minYear)

Minimum year is : 1922.0


In [41]:
millionsongs_df = millionsongs_raw_df
millionsongs_df['year'] = millionsongs_raw_df['year'] - minyear

In [42]:
millionsongs_df.head()

,year,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12
0,79.0,0.884124,0.610454,0.600498,0.474669,0.247233,0.357306,0.344136,0.339641,0.600859,0.425705,0.604915,0.419193
1,79.0,0.854412,0.604125,0.593634,0.495885,0.266308,0.261472,0.506387,0.464454,0.665799,0.542969,0.580444,0.445219
2,79.0,0.908983,0.632063,0.557429,0.498264,0.276396,0.312810,0.448530,0.448674,0.649791,0.489869,0.591908,0.450002
3,79.0,0.842525,0.561827,0.508715,0.443531,0.296734,0.250214,0.488541,0.360509,0.575435,0.361006,0.678379,0.409037
4,79.0,0.909303,0.653608,0.585581,0.473251,0.251417,0.326977,0.404323,0.371155,0.629402,0.482243,0.566901,0.463374


In [43]:
features_list = ['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12']
target = ['year']

In [44]:
millionsongs_df_features = millionsongs_df[features_list]
millionsongs_df_target = millionsongs_df[target]

In [51]:
millionsongs_df_features.shape

(6724, 12)

#### Create interaction features and Polynomial features

Perform feature expansion in a polynomial space. As said in wikipedia of Polynomial Expansion, which is available at http://en.wikipedia.org/wiki/Polynomial_expansion, “In mathematics, an expansion of a product of sums expresses it as a sum of products by using the fact that multiplication distributes over addition”. Take a 2-variable feature vector as an example: (x, y), if we want to expand it with degree 2, then we get (x, x * x, y, x * y, y * y).


In [46]:
from sklearn import preprocessing
poly = preprocessing.PolynomialFeatures(2)
millionsongs_poly_df_features = poly.fit_transform(millionsongs_df_features)

In [50]:
millionsongs_poly_df_features.shape

(6724, 91)

#### Create Train, Test and Validation datasets

In [52]:
from sklearn.cross_validation import train_test_split

In [53]:
train_df_features, test_df_features, train_df_target, test_df_target = train_test_split(millionsongs_df_features, 
                                                                                        millionsongs_df_target,
                                                                                        test_size = 0.3,
                                                                                        random_state = 10)

In [54]:
test_df_features, valid_df_features, test_df_target, valid_df_target = train_test_split(test_df_features, 
                                                                                        test_df_target,
                                                                                        test_size = 0.5,
                                                                                        random_state = 11)

In [58]:
print train_df_features.shape
print train_df_target.shape
print test_df_features.shape
print test_df_target.shape
print valid_df_features.shape
print valid_df_target.shape

(4706, 12)
(4706, 1)
(1009, 12)
(1009, 1)
(1009, 12)
(1009, 1)


## Part 3: Create a Baseline model

In this section, we will create a baseline model and evaluate that model with validation and test datasets. 

* Create an average model
* Calculate the RMSE on the validation and test datasets based on the predictions by average model

#### Average model

A very simple yet natural baseline model is one where we always make the same prediction independent for a given data point, using the average label in the training set as the constant prediction value. Compute this value, which is the average(shifted) song year for the training set.

In [61]:
averageTrainYear = train_df_target.year.mean()
print averageTrainYear

54.0342116447


#### Root mean squared error

We naturally would like to see how well this naive baseline performs. We will use root mean squared error(RMSE) for evaluation purposes.

For this we can use sklearn.metrics.mean_squared_error

In [62]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [63]:
labelsAndPredsTrain_base = pd.DataFrame(train_df_target.year)
labelsAndPredsVal_base = pd.DataFrame(valid_df_target.year)
labelsAndPredsTest_base = pd.DataFrame(test_df_target.year)

In [64]:
labelsAndPredsTrain_base['year_pred'] = averageTrainYear
labelsAndPredsVal_base['year_pred'] = averageTrainYear
labelsAndPredsTest_base['year_pred'] = averageTrainYear

In [65]:
labelsAndPredsTrain_base.head()

,year,year_pred
2526,65.0,54.034212
6074,33.0,54.034212
313,76.0,54.034212
2387,44.0,54.034212
6196,31.0,54.034212


In [66]:
labelsAndPredsVal_base.head()

,year,year_pred
5929,33.0,54.034212
3954,59.0,54.034212
1033,74.0,54.034212
3806,58.0,54.034212
5709,36.0,54.034212


In [67]:
labelsAndPredsTest_base.head()

,year,year_pred
5619,37.0,54.034212
784,48.0,54.034212
1651,74.0,54.034212
6519,31.0,54.034212
4791,39.0,54.034212


In [68]:
baseline_mse_train = mean_squared_error(labelsAndPredsTrain_base.year, labelsAndPredsTrain_base.year_pred)
baseline_rmse_train = sqrt(baseline_mse_train)
baseline_mse_val = mean_squared_error(labelsAndPredsVal_base.year, labelsAndPredsVal_base.year_pred)
baseline_rmse_val = sqrt(baseline_mse_val)
baseline_mse_test = mean_squared_error(labelsAndPredsTest_base.year, labelsAndPredsTest_base.year_pred)
baseline_rmse_test = sqrt(baseline_mse_test)

In [69]:
print "baseline_rmse_train is : "+ str(baseline_rmse_train)
print "baseline_rmse_val is : "+ str(baseline_rmse_val)
print "baseline_rmse_test is : "+ str(baseline_rmse_test)

baseline_rmse_train is : 21.2946538004
baseline_rmse_val is : 21.8597131291
baseline_rmse_test is : 21.556355903


## Part 4: Train a model using Linear Regression 

We now have some idea about the validation and test errors from baseline. But we should be able to do a little better by using a linear regression model.

Stochastic Gradient Descent Regression

In [79]:
from sklearn import linear_model

In [80]:
model1 = linear_model.SGDRegressor()

In [81]:
model1.fit(train_df_features,train_df_target)

/data1/software/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', n_iter=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, verbose=0, warm_start=False)

In [82]:
model1.intercept_

array([ 14.06204221])

In [83]:
model1.coef_

array([ 25.12928262,  24.76020184,  -9.43253355,   7.24264089,
         4.7574048 , -14.95139095,  19.08740424,  -1.3599755 ,
         6.30522312,   2.02558312,   9.37962069,  -2.06477785])

Make Predictions on the validation data and test data

#### Evaluate RMSE

Next evaluate the accuracy of this model on the validation set. Use the predict() method to create lablesAndPreds

In [89]:
train_df_preds_model1 = model1.predict(train_df_features)
valid_df_preds_model1 = model1.predict(valid_df_features)
test_df_preds_model1 = model1.predict(test_df_features)

In [90]:
labelsAndPredsTrain_model1 = pd.DataFrame(train_df_preds_model1, columns=['year_pred'])
labelsAndPredsVal_model1 = pd.DataFrame(valid_df_preds_model1, columns=['year_pred'])
labelsAndPredsTest_model1 = pd.DataFrame(test_df_preds_model1, columns=['year_pred'])
labelsAndPredsTrain_model1['year_true'] = train_df_target
labelsAndPredsVal_model1['year_true'] = valid_df_target
labelsAndPredsTest_model1['year_true'] = test_df_target

In [92]:
model1_rmse_train = sqrt(mean_squared_error(train_df_target.year,labelsAndPredsTrain_model1.year_pred))
model1_rmse_valid = sqrt(mean_squared_error(valid_df_target.year,labelsAndPredsVal_model1.year_pred))
model1_rmse_test = sqrt(mean_squared_error(test_df_target.year,labelsAndPredsTest_model1.year_pred))

In [93]:
print "model1_rmse_train is : "+ str(model1_rmse_train)
print "model1_rmse_val is : "+ str(model1_rmse_valid)
print "model1_rmse_test is : "+ str(model1_rmse_test)

model1_rmse_train is : 17.4993351591
model1_rmse_val is : 17.8211026979
model1_rmse_test is : 17.6660567835


## Part 5: Choose the best model by Hyperparamter tuning


* Perform grid search fo find a good regularization parameter, number of iterations, initial step size
* Report the test dataset error using the best model selected using the Hyperparameter optimization.

We are already outperforming the baseline on the validation setby almost 5 years on average, but let's see if we can do better by selecting a best model based with hyperparameter tuning. 

Hyperparameter Tuning/Optimization or model selection is the problem of choosing a set of hyperparameters for a learning algorithm, usually with the goal of optimizing a measure of the algorithm's performance on an independent data set such as Validation dataset. 
https://en.wikipedia.org/wiki/Hyperparameter_optimization


For this model selection, the hyperparameters we are tuning are regularization parameter, initial step size, number of iterations. Regularization type we are using is L2 norm.

#### Perform grid search fo find a good regularization parameter, number of iterations, initial step size

In [94]:
parameters = {"alpha":[1e-10,1e-5,1e0,1e1,1e10], 
              "eta0":[0.01,0.1,1],
              "n_iter":[1,5,10,100]}

In [96]:
from sklearn import grid_search
from sklearn import metrics

In [97]:
mse_scorer = metrics.make_scorer(metrics.mean_squared_error, greater_is_better = False)

In [105]:
sgdmodel = grid_search.GridSearchCV(linear_model.SGDRegressor(), scoring=mse_scorer, param_grid=parameters, cv=10)

In [106]:
sgdmodel.fit(train_df_features,train_df_target)

/data1/software/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/data1/software/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/data1/software/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/data1/software/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

GridSearchCV(cv=10, error_score='raise',
       estimator=SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', n_iter=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': [1e-10, 1e-05, 1.0, 10.0, 10000000000.0], 'eta0': [0.01, 0.1, 1], 'n_iter': [1, 5, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring=make_scorer(mean_squared_error, greater_is_better=False),
       verbose=0)

In [107]:
sgdmodel.best_estimator_

SGDRegressor(alpha=1e-10, average=False, epsilon=0.1, eta0=0.1,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', n_iter=100, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, verbose=0, warm_start=False)

In [108]:
sgdmodel.best_params_

{'alpha': 1e-10, 'eta0': 0.1, 'n_iter': 100}

In [109]:
train_df_preds_sgdmodel = sgdmodel.best_estimator_.predict(train_df_features)
valid_df_preds_sgdmodel = sgdmodel.best_estimator_.predict(valid_df_features)
test_df_preds_sgdmodel = sgdmodel.best_estimator_.predict(test_df_features)

In [110]:
labelsAndPredsTrain_sgdmodel = pd.DataFrame(train_df_preds_sgdmodel, columns=['year_pred'])
labelsAndPredsVal_sgdmodel = pd.DataFrame(valid_df_preds_sgdmodel, columns=['year_pred'])
labelsAndPredsTest_sgdmodel = pd.DataFrame(test_df_preds_sgdmodel, columns=['year_pred'])

In [113]:
sgdmodel_rmse_train = sqrt(mean_squared_error(train_df_target.year,labelsAndPredsTrain_sgdmodel.year_pred))
sgdmodel_rmse_val = sqrt(mean_squared_error(valid_df_target.year,labelsAndPredsVal_sgdmodel.year_pred))
sgdmodel_rmse_test = sqrt(mean_squared_error(test_df_target.year,labelsAndPredsTest_sgdmodel.year_pred))

In [114]:
print "Best Model sgdmodel_rmse_train is : "+ str(sgdmodel_rmse_train)
print "Best Model sgdmodel_rmse_val is : "+ str(sgdmodel_rmse_val)
print "Best Model sgdmodel_rmse_test is : "+ str(sgdmodel_rmse_test)

Best Model sgdmodel_rmse_train is : 15.7108595792
Best Model sgdmodel_rmse_val is : 16.3577722487
Best Model sgdmodel_rmse_test is : 15.7214429183


#### Report the test dataset error using the best model selected using the Hyperparameter optimization.

In [117]:
print "Average Baseline Model Test Error is : "+ str(baseline_rmse_test)
print "Linear Regression Model1 Test Error is : "+ str(model1_rmse_test)
print "Linear Regression Best Model Test Error is : "+ str(sgdmodel_rmse_test)

Average Baseline Model Test Error is : 21.556355903
Linear Regression Model1 Test Error is : 17.6660567835
Linear Regression Best Model Test Error is : 15.7214429183


** As we can see from the above results, we were able to improve our prediction results from the baseline average model by a huge margin.**


** Further we can build additional models using the polynomial features to improve the model's predictions and reduce test errorts**
